# Read table from database

In [ ]:
import os
# Importing YData's packages
from ydata.labs import Connectors
# Getting a previously created Connector
connector = Connectors.get(uid='{connector-id}',
                           namespace='{namespace-id}')
print(connector)

In [21]:
table = os.getenv('table', 'dimcustomer')

In [22]:
dataset = connector.connector.query(f"SELECT * FROM {table};")
df = dataset.to_pandas()
sample = df.head(10)

## Calculating the Metadata & the Profile

In [23]:
from ydata.metadata import Metadata

if table == 'dimproduct':
    cols = ['productkey', 'productalternatekey', 'englishproductname', 'standardcost','listprice', 'dealerprice','status', 'startdate', 'enddate']
    df=df[cols]
elif table == 'factinternetsales':
    #prep the original dataset
    cols=['unitprice','productstandardcost', 'totalproductcost', 'salesamount', 'taxamt','extendedamount', 'freight']

    for col in cols:
        df[col]=df[col].str.replace('$', '').str.replace(',', '').astype('float')
else:
    cols = ['yearlyincome']
    for col in cols:
        df[col]=df[col].str.replace('$', '').str.replace(',', '').astype('float')

m = Metadata(dataset)

/tmp/ipykernel_502/926253412.py:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col]=df[col].str.replace('$', '').str.replace(',', '').astype('float')


In [24]:
m.save(f'metadata_{table}.pkl')

In [25]:
from ydata_profiling import ProfileReport

report = ProfileReport(df, minimal=True)
report_html = report.to_html()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [26]:
df.to_csv(f'{table}.csv', index=False)

## Setting the pipeline step outputs

In [27]:
##add here the outputs logic
import json

profile_pipeline_output = {
    'outputs' : [
        {
            'type': 'table',
            'storage': 'inline',
            'format': 'csv',
            'header': list(sample.columns),
            'source': sample.to_csv(header=False, index=True)
        },
        {
          'type': 'web-app',
          'storage': 'inline',
          'source': report_html,
        },
    ]
  }

with open('mlpipeline-ui-metadata.json', 'w') as metadata_file:
    json.dump(profile_pipeline_output, metadata_file)
